In [1]:
!pip install --upgrade azure-cognitiveservices-vision-computervision

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.9 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.6/178.6 kB 3.0 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.5/151.5 kB 2.3 MB/s eta 0:00:0031m21.8 MB/s eta 0:00:01


In [2]:
!pip install pillow

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Defaulting to user installation because normal site-packages is not writeable


In [10]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import sys
import time

import json

In [11]:
with open('secret.json') as f:
    secret = json.load(f)

In [12]:
KEY = secret['KEY']
ENDPOINT = secret['ENDPOINT']

In [13]:
computervision_client = ComputerVisionClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [14]:
remote_image_url = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-sample-data-files/master/ComputerVision/Images/landmark.jpg"

## 画像の説明

In [15]:
print("===== Describe an image - remote =====")
description_results = computervision_client.describe_image(remote_image_url )

print("Description in the remote image: ")
if (len(description_results.tags) == 0):
    print("No description detected.")
else:
    for caption in description_results.captions:
        print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))

===== Describe an image - remote =====
Description in the remote image: 
'an ancient city with many ruins with Colosseum in the background' with confidence 33.80%


## 画像カテゴリの取得

In [16]:
print("===== Categorize an Image - remote =====")
remote_image_features = ["categories"]
categorize_results_remote = computervision_client.analyze_image(remote_image_url, remote_image_features)

print("Categories from remote image: ")
if (len(categorize_results_remote.categories) == 0):
    print("No categories detected.")
else:
    for category in categorize_results_remote.categories:
        print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))

===== Categorize an Image - remote =====
Categories from remote image: 
'building_' with confidence 31.64%
'others_' with confidence 0.39%
'outdoor_' with confidence 3.91%


## 画像タグの取得

In [17]:
print("===== Tag an image - remote =====")
tags_result_remote = computervision_client.tag_image(remote_image_url )

print("Tags in the remote image: ")
if (len(tags_result_remote.tags) == 0):
    print("No tags detected.")
else:
    for tag in tags_result_remote.tags:
        print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))

===== Tag an image - remote =====
Tags in the remote image: 
'outdoor' with confidence 99.00%
'building' with confidence 98.81%
'sky' with confidence 98.21%
'stadium' with confidence 98.17%
'ancient rome' with confidence 96.16%
'ruins' with confidence 95.04%
'amphitheatre' with confidence 93.99%
'ancient roman architecture' with confidence 92.65%
'historic site' with confidence 89.55%
'ancient history' with confidence 89.54%
'history' with confidence 86.72%
'archaeological site' with confidence 84.41%
'travel' with confidence 65.85%
'large' with confidence 61.02%
'city' with confidence 56.57%


## 物体を検出

In [18]:
print("===== Detect Objects - remote =====")
remote_image_url_objects = "https://azurecomcdn.azureedge.net/cvt-493701653c65826061dc17c66113eb5d9e70cd831a30707034222f935a1a140d/images/shared/cognitive-services-demos/analyze-image/analyze-3-thumbnail.jpg"
detect_objects_results_remote = computervision_client.detect_objects(remote_image_url )

print("Detecting objects in remote image:")
if len(detect_objects_results_remote.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results_remote.objects:
        print("object at location {}, {}, {}, {}".format( \
        object.rectangle.x, object.rectangle.x + object.rectangle.w, \
        object.rectangle.y, object.rectangle.y + object.rectangle.h))

===== Detect Objects - remote =====
Detecting objects in remote image:
No objects detected.


## ローカルファイルに対応させる

In [22]:
local_image_path = 'sample01.jpg'
local_image = open(local_image_path, "rb")

print("===== Detect Objects - local =====")
detect_objects_results = computervision_client.detect_objects_in_stream(local_image )

print("Detecting objects in remote image:")
if len(detect_objects_results.objects) == 0:
    print("No objects detected.")
else:
    for object in detect_objects_results.objects:
        print("object at location {}, {}, {}, {}".format( \
        object.rectangle.x, object.rectangle.x + object.rectangle.w, \
        object.rectangle.y, object.rectangle.y + object.rectangle.h))

===== Detect Objects - local =====
Detecting objects in remote image:
object at location 871, 1204, 258, 779
object at location 423, 1086, 829, 1257
object at location 1258, 1746, 752, 1291
object at location 30, 1616, 177, 1434


In [65]:
def get_tags(filepath):
    local_image = open(filepath, "rb")

    tags_result = computervision_client.tag_image_in_stream(local_image )
    tags = tags_result.tags
    tags_name = []
    for tag in tags:
        tags_name.append(tag.name)
    return tags_name

In [66]:
filepath = 'sample01.jpg'
get_tags(filepath)

['tableware',
 'food',
 'baked goods',
 'plate',
 'drink',
 'coffee cup',
 'dishware',
 'saucer',
 'snack',
 'serveware',
 'meal',
 'mug',
 'tea',
 'fast food',
 'breakfast',
 'fork',
 'dish',
 'kitchen utensil',
 'brunch',
 'platter',
 'dessert',
 'cup',
 'coffee',
 'indoor',
 'sitting',
 'table']

In [57]:
def detect_objects(filepath):
    local_image = open(filepath, "rb")

    detect_objects_results = computervision_client.detect_objects_in_stream(local_image )
    objects = detect_objects_results.objects
    return objects

In [59]:
filepath = 'sample01.jpg'
objects = detect_objects(filepath)

In [64]:
objects[0].rectangle.x

871